In [1]:
# Importing the usual stuff

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import math
import numpy as np

import datetime

import plotly.express as px
from plotly.subplots import make_subplots

pd.set_option('display.max_columns', None)

In [38]:
#  Getting the dataframe that links every product to its description and brand

df_sku_master = pd.read_excel('MASTER_SKU_BRAND.xlsx')
df_sku_master['SKU family'] = df_sku_master['ITEM_NAME'].apply(lambda x: x[:-2])
df_sku_master.drop_duplicates(subset='SKU family',keep='first',inplace=True)

In [41]:
# Getting the dataframe that links every product to its type

df_merch_type = pd.read_excel(r'C:\Users\btg168\Downloads\export_merch_type.xlsx')
df_merch_type['CODE_DESC'].replace({"L'Oreal LUXE Make up":"MAKE-UP","L'Oreal LUXE Skin Care":"SKIN CARE",\
                                    "L'Oreal LUXE Hair Care":"HAIR CARE","HAIR":"HAIR CARE",\
                                    "L'Oreal LUXE Miscellances":"MISCELLANEOUS"},inplace=True)
df_merch_type['SKU family'] = df_merch_type['ITEM_NAME'].apply(lambda x: x[:-2])
df_merch_type.drop_duplicates(subset='SKU family',keep='first',inplace=True)

In [42]:
# Taking a look at the different product type

df_merch_type['CODE_DESC'].unique()

array(['MAKE-UP', 'MISCELLANEOUS', 'SKIN CARE',
       "L'Oreal LUXE Beauty Dummy SKU", 'FRAGRANCE',
       "L'Oreal LUXE Perfume", 'HYGIENE', 'HAIR CARE',
       "L'Oreal LUXE promotional Object & packaging"], dtype=object)

In [5]:
# Getting the dataframe of claims 'df_raw'

df_raw = pd.read_excel(r'C:\Users\btg168\Downloads\Customer Complaint - 2020 UPDATED.xlsx')

In [6]:
# Cleaning 'df_raw'

new_headers = df_raw.iloc[3]
df_raw = df_raw.iloc[4:]
df_raw.columns = new_headers
df_raw.columns.name = None
df_raw.dropna(thresh = 5, inplace = True)
df_raw['Complaint Date'] = pd.to_datetime(df_raw['Complaint Date'], format = '%Y/%m/%d')
df_raw.dropna(thresh = 5, axis = 1, inplace = True)
df_raw.rename(columns = {'Others\nDiscrepancies ':'category'}, inplace = True)
df_raw.replace('ySL','YSL', inplace = True)

In [7]:
#Checking the missing values

df_raw.isna().sum()

No.                                                0
Complaint Date                                     0
Week                                               0
Month                                              0
#Outbound \nErrors                                 0
Loreal CS                                          0
Consignee                                          0
Shipment #                                         0
Shipment Type\n(Sea/Air/Truck/Local pick up)       0
DO #                                               0
ETD                                                0
SKU#                                               0
Product \nType                                     0
Brand                                              0
Division                                           1
Claim Qty                                          0
COUNTRY OF ORIGIN                                  1
Total Amount                                       1
Currency                                      

In [8]:
# Checking the values of 'category'

df_raw['category'].unique()

array(['SHORTAGE', 'DAMAGED', 'EXCESS', 'WHITE SCRATCH', 'OLD BATCH',
       'TESTER', 'PACKAGING ISSUE', 'CHINESE LANGUAGE'], dtype=object)

In [9]:
# Creating the final dataframe that we will be using for the analysis

df = df_raw.copy()
#working only with 'DAMAGED' claims
df = df.loc[df['category'] == 'DAMAGED']
#working only with 'Complaint date' from 2020
df = df[df['Complaint Date'].dt.year == 2020]
df.reset_index(inplace = True, drop = True)
df['SKU family'] = df['SKU#'].apply(lambda x: x[:-2])
df['Total Amount'] = round(df['Total Amount'].astype(float), 1)
df.rename(columns = {df.columns[8]:'Shipment'}, inplace = True)
df['Total Amount SGD'] = df['Total Amount']
df.loc[df['Currency'] == 'THD','Total Amount SGD'] = df['Total Amount'].apply(lambda x: x*0.044)
df.loc[df['Currency'] == 'JPY','Total Amount SGD'] = df['Total Amount'].apply(lambda x: x*0.013)
df.loc[df['Currency'] == 'HKD','Total Amount SGD'] = df['Total Amount'].apply(lambda x: x*0.18)
df.loc[df['Currency'] == 'USD','Total Amount SGD'] = df['Total Amount'].apply(lambda x: x*1.37)
df = df.merge(df_merch_type, on = 'SKU family', how = 'left')
df.drop_duplicates(subset = ['Complaint Date','Loreal CS','Shipment #','DO #','SKU#','Claim Qty '],\
                   keep = 'first', inplace = True)
df = df.merge(df_sku_master[['SKU family','DESCRIPTION']].\
              drop_duplicates(subset = ['SKU family'], keep = 'first'), on = 'SKU family', how = 'left')

In [37]:
df.head()

,No.,Complaint Date,Week,Month,#Outbound \nErrors,Loreal CS,Consignee,Shipment #,Shipment,DO #,ETD,SKU#,Product \nType,Brand,Division,Claim Qty,COUNTRY OF ORIGIN,Total Amount,Currency,category,Pick from \npallet / shelving,Bollore finding reponse date,Responsible party,Finding,Root Cause,Order type,simplified packing,Claims \nstatus,Corrective \nMeasure,Preventive \nMeasure,REMARKS,SKU family,Total Amount SGD,ITEM_NAME,CODE_DESC,DESCRIPTION
0,19,2020-01-02,1,1,1,LAM LOUIE,EVER RICH D.F.S.CORPORATION,48758,SEA,80174836,2019-11-21 00:00:00,L4082002,FINISHED PRODUCTS,GIORGIO ARMANI,LLD,3,FR,51.4,USD,DAMAGED,S,2020-01-06,NaN,Investigation has shown no damaged carton / pa...,NO FINDING,FG,NaN,Rejected,NaN,NaN,NaN,L40820,70.418,L4082000,MAKE-UP,LUMINOUS SILK 2 F/P30ML
1,20,2020-01-02,1,1,1,LAM LOUIE,EVER RICH D.F.S.CORPORATION,48758,SEA,80174836,2019-11-21 00:00:00,L2785503,FINISHED PRODUCTS,YSL,LLD,4,FR,60.5,USD,DAMAGED,S,2020-01-06,NaN,Investigation has shown no damaged carton / pa...,NO FINDING,FG,NaN,Rejected,NaN,NaN,NaN,L27855,82.885,L2785500,MAKE-UP,RPC VERNIS A LEVRES 09
2,21,2020-01-02,1,1,1,LAM LOUIE,EVER RICH D.F.S.CORPORATION,48758,SEA,80174836,2019-11-21 00:00:00,L8958000,FINISHED PRODUCTS,YSL,LLD,2,FR,31.9,USD,DAMAGED,S,2020-01-06,NaN,Investigation has shown no damaged carton / pa...,NO FINDING,FG,NaN,Rejected,NaN,NaN,NaN,L89580,43.703,L8958000,MAKE-UP,VOLUPTE PLUMP-IN-COLOUR 4
3,22,2020-01-02,1,1,1,LAM LOUIE,EVER RICH D.F.S.CORPORATION,48758,SEA,80174836,2019-11-21 00:00:00,L8958100,FINISHED PRODUCTS,YSL,LLD,1,FR,16.0,USD,DAMAGED,S,2020-01-06,NaN,Investigation has shown no damaged carton / pa...,NO FINDING,FG,NaN,Rejected,NaN,NaN,NaN,L89581,21.920,L8958100,MAKE-UP,VOLUPTE PLUMP-IN-COLOUR 5
4,23,2020-01-02,1,1,1,LAM LOUIE,EVER RICH D.F.S.CORPORATION,48758,SEA,80174839,2019-11-21 00:00:00,M9066000,FINISHED PRODUCTS,LA ROCHE POSAY,ACD / PPD,4,FR,34.2,USD,DAMAGED,S,2020-01-06,NaN,Investigation has shown no damaged carton / pa...,NO FINDING,FG,NaN,Rejected,NaN,NaN,NaN,M90660,46.854,M9066000,SKIN CARE,EFFACLAR MAT T40ML F/EN
